In [1]:
import pandas as pd
import time
import datetime
import numpy as np
import split_data
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl  
import helper
import warnings
warnings.filterwarnings('ignore')
import plot_show
import tslearn_cluster
from sklearn.preprocessing import MinMaxScaler

Loading BokehJS ...

In [2]:
data_df = pd.read_csv('../data/zhao_neng_2_month.txt',sep='\t',header=None,names=['ap_mac','timestamp','count'])
types_df = pd.read_csv("../data/eqp_type.txt",sep='\t',names=['ap_mac','company','address','type'])
print('原始数据ap个数',len(data_df.ap_mac.unique()))

原始数据ap个数 60599


In [3]:
df_with_type = data_df.merge(types_df,how='left',on='ap_mac')
df_with_type.drop(columns=['company','address'],inplace=True)
df_with_type.fillna(0,inplace=True)
df_with_type.type = pd.to_numeric(df_with_type.type, downcast='signed')

In [ ]:
df_with_type.head()

,ap_mac,timestamp,count,type
0,08-A5-C8-28-21-DB,1521432000,28,0
1,08-A5-C8-28-21-DB,1521435600,25,0
2,08-A5-C8-28-21-DB,1521439200,34,0
3,08-A5-C8-28-21-DB,1521442800,30,0
4,08-A5-C8-28-21-DB,1521446400,33,0


In [ ]:
# 获取所有类型的24小时平均值
def get_all_type_24h():
    types = df_with_type.type.unique()
    all_type_24 = []
    all_type_num = []
    for type_one in types:
        df = df_with_type[df_with_type.type==type_one]
        macs = df.ap_mac.unique()
        all_type_num.append(len(macs))
        df = helper.get_split_ts(df,macs)
        scaler = MinMaxScaler()
        scaler_data = scaler.fit_transform(df)
        type_one_means = scaler_data.mean(axis=1)
        all_type_24.append(type_one_means)
    return all_type_24,all_type_num
all_type_24,all_num = get_all_type_24h()

In [ ]:
def one_type(type_class):
    df = df_with_type[df_with_type.type==type_class]
    macs = df.ap_mac.unique()
    df = helper.get_split_ts(df,macs)
    scaler = MinMaxScaler()
    scaler_data = scaler.fit_transform(df)
    plot_show.plt_one_ts(scaler_data.mean(axis=1))   
# one_type(12)#购物广场